# TimeGPT fine-tuning validation (real API)

Validates TimeGPT zero-shot vs persistent fine-tuning using the **real Nixtla API** and the same dataset as the Chronos-family notebook.

1. Load dataset (events_pageviews)
2. Train/test split (last h=14 as test)
3. Zero-shot forecast
4. Persistent fine-tune with `fit()`
5. Forecast with fine-tuned model
6. Evaluate MAE (zero-shot vs finetuned)

**Requires:** `NIXTLA_API_KEY` set in the environment.

In [ ]:
import os

if "NIXTLA_API_KEY" not in os.environ:
    raise ValueError("Set NIXTLA_API_KEY environment variable before running")

In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error

from timecopilot.models.foundation.timegpt import TimeGPT

## Load dataset

Same source as chronos-family notebook. Columns: `unique_id`, `ds`, `y`.

In [ ]:
df = pd.read_csv(
    "https://timecopilot.s3.amazonaws.com/public/data/events_pageviews.csv",
    parse_dates=["ds"],
)
df["unique_id"] = df["unique_id"].astype(str)

first_id = df["unique_id"].iloc[0]
times = df.loc[df["unique_id"] == first_id, "ds"].sort_values()
freq = pd.infer_freq(times.values) or "D"

print("freq:", freq)
df.head()

## Train/test split

For each series: train = all except last h rows, test = last h rows.

In [ ]:
h = 14
train = (
    df.groupby("unique_id", group_keys=False)
    .apply(lambda g: g.iloc[:-h])
    .reset_index(drop=True)
)
test = (
    df.groupby("unique_id", group_keys=False)
    .apply(lambda g: g.iloc[-h:])
    .reset_index(drop=True)
)
print("train:", train.shape, "test:", test.shape)

## Zero-shot forecast

In [ ]:
m0 = TimeGPT()
fcst0 = m0.forecast(train, h=h, freq=freq)
fcst0.head()

## Fine-tuning (persistent)

In [ ]:
m1 = TimeGPT()
ft_id = m1.fit(
    train,
    freq=freq,
    finetune_steps=50,
    finetune_depth=1,
    finetune_loss="default",
)
print("Finetuned model id:", ft_id)

## Forecast using fine-tuned model

In [ ]:
fcst1 = m1.forecast(train, h=h, freq=freq)
fcst1.head()

## Evaluation (MAE)

In [ ]:
merged0 = test.merge(
    fcst0[["unique_id", "ds", m0.alias]],
    on=["unique_id", "ds"],
)
merged1 = test.merge(
    fcst1[["unique_id", "ds", m1.alias]],
    on=["unique_id", "ds"],
)

mae0 = mean_absolute_error(merged0["y"], merged0[m0.alias])
mae1 = mean_absolute_error(merged1["y"], merged1[m1.alias])

print("Zero-shot MAE:", mae0)
print("Finetuned MAE:", mae1)